In [3]:
import pandas as pd
import numpy as np
import calendar
import dateutil
from dateutil import parser

years = [2018,2019]
#years = [2019]
data_frames_list = []

df = pd.read_csv('data//Statements_Midata.csv', encoding='ISO-8859-1', sep=';') # Read file to process into memory
df = df.rename(columns={'Debit/Credit': 'Amount', 'Type': 'Transaction', 'Merchant/Description': 'Desc'}) # Rename columns to something more useful

df['Date'] = pd.to_datetime(df.Date, dayfirst=True) # Deal with UK format dates
df.loc[df['Balance'].str.contains('\+£'), ['Balance']] = df['Balance'].str.replace("\+£", "")
df['Debit'] = df.loc[df['Amount'].str.contains('-£'), ['Debit']] = df['Amount'].str.replace("-£", "")
df['Credit'] = df.loc[df['Amount'].str.contains('\+£'), ['Credit']] = df['Amount'].str.replace("\+£", "")

df['Balance'] = pd.to_numeric(df['Balance'], errors='coerce')
df['Debit'] = pd.to_numeric(df['Debit'], errors='coerce')
df['Credit'] = pd.to_numeric(df['Credit'], errors='coerce')
df = df.drop(columns=['Amount',',,,'])

for year in years:
    for month in range(1, 13):
        start_date = parser.parse(str(calendar.month_name[month]) + " " + str('01') + " " + str(year)) 
        end_date = parser.parse(str(calendar.month_name[month]) + " " + str(calendar.monthrange(year, month)[1]) + " " + str(year)) 
        mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
        #mask = (df['Date'] >= start_date) & (df['Date'] <= end_date) & (df['Transaction'] == 'DD')
        data = df.loc[mask].sort_values('Date', ascending=True)
        #dd = df.loc[df['Transaction'] == 'DD']
        #data = data.loc[data['Transaction'] == 'DD']
        if data.empty == False:
            data.loc['Total',4:6] = data.sum(axis=0)
            data.loc['Total','Surplus/Deficit'] = data.loc['Total', 'Credit'] - data.loc['Total', 'Debit']
            data_frames_list.append(data)
final = pd.concat(data_frames_list)
try:
  final.to_csv('data//transactions_by_month.csv')
except:
  print("Something is wrong. Is the excel file open?")

In [468]:
df


,Date,Transaction,Desc,Balance,Debit,Credit
